Copy from following link: https://www.kaggle.com/code/kingabzpro/gemma-7b-simple-inference-on-gpu
        
Other set ups have worked, but this has the lowest inference latency.
Plan now is to: <br>

__1.0 Hold: Add Gradio UI__ .... Don't think Gradio is very customisable + still having issues getting going. Might be easier to stick with fastapi/Uvicorn + Jijna/HTML

__2.0 Add RAG__ (based on 4. Vertex deploy...)

__3.0 UI for Playground__ (see 1.0) will be critical. 1st just need Q&A, then can expand to ingesting docs, but that adds obvious next steps (UI to support + backend to process)

### -- Existing code from above link

In [2]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

modelName = "/kaggle/input/gemma/transformers/7b-it/2"

bnbConfig = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(modelName)

model = AutoModelForCausalLM.from_pretrained(
    modelName,
    device_map = "auto",
    quantization_config=bnbConfig
)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
from IPython.display import Markdown, display
system =  "You are a skilled software engineer who consistently produces high-quality Python code."
user = "Write a Python code to display text in a star pattern."

prompt = f"System: {system} \n User: {user} \n AI: "
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=500, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

Markdown(text.split("AI:")[1])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
2024-04-26 03:38:45.746078: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-26 03:38:45.746205: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-26 03:38:45.909688: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


 

```python
def star_pattern(n):
    for row in range(n):
        for col in range(n):
            if (col == 0 or row == n - 1) and (row >= 1 and col >= 1):
                print("*", end=" ")
            else:
                print(" ", end=" ")
        print()

star_pattern(5)
```

**Explanation:**

* The function `star_pattern` takes an integer `n` as input, representing the number of rows in the star pattern.
* It uses nested loops to iterate over the rows and columns of the pattern.
* The condition `(col == 0 or row == n - 1) and (row >= 1 and col >= 1)` checks if the current position is at the first column or the last row, and if the row and column positions are greater than or equal to 1.
* If the condition above is true, it prints a star (`*`) followed by a space. Otherwise, it prints a space.
* The loop prints a newline character (`\n`) after each row of stars is printed.
* Finally, the `star_pattern` function is called with an input of 5, which will produce the following star pattern:

```
    *
   ***
  *****
 *******
```

**Note:**

* The code assumes that the input `n` will be an integer.
* The number of stars printed in each row increases by one from the previous row.
* The spacing between the stars is one space.
* The stars are printed in the center of the row.

### -- RAG set up

In [5]:
# --- Install lib
#!pip3 install --upgrade google-cloud-aiplatform
#!pip3 install ipython pandas[output_formatting] google-cloud-language==2.10.0
!pip install langchain
!pip install PyPDF2
!pip install pypdf
!pip install sentence_transformers
!pip install torch
!pip install pandas
!pip install pdfplumber

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 797.6 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 5.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are ins

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.7 MB/s eta 0:00:00a 0:00:01


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.8 MB/s eta 0:00:00a 0:00:01m


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 16.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 18.9 MB/s eta 0:00:0000:0100:01


In [73]:
# --- Ingest and chunk data (now using new doc ingestion)

# Uses same loader as above, but now works for multiple docs
#from langchain.document_loaders import PDFPlumberLoader
import os as os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pdfplumber
from langchain.document_loaders import PDFPlumberLoader

def load_pdfs_from_directory(directory_path):
    docs = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory_path, filename)
            loader = PDFPlumberLoader(file_path)
            loaded_docs = loader.load()
            
            #text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=250)
            
            for doc in loaded_docs:
                texts = text_splitter.split_text(doc.page_content)
                for text in texts:
                    docs.append({
                        'page_content': text,
                        'metadata': doc.metadata
                    })
    return docs

#directory_path = "/home/jupyter/pdf"
directory_path = "/kaggle/input/benpayfactsandcs/"
all_pdf_docs = load_pdfs_from_directory(directory_path)

data = all_pdf_docs
df = pd.DataFrame(data)


df['len'] = df['page_content'].str.len()

df1 = df.copy()

df1['page_content_data'] = df1['page_content']
df1['page_content'] = df1['page_content'].str.replace('\n', '')

df2 = pd.json_normalize(df1.metadata)
df1 = pd.concat([df1, df2], axis=1)
df1['source'] = df1['source'].str.replace('/kaggle/input/benpayfactsandcs/', '')
#/kaggle/input/benpayfactsandcs/bendigo-payment-facilities-terms-conditions.pdf

df1['pagestr'] = df1['page'].astype(str)

df1['page_content_plus'] = df1['page_content'] + '[' + 'Source: ' + df1['source'] + ' page: ' + df1['pagestr'] + ']'

#df1.drop(['metadata'], axis=1, inplace=True)
# df1.drop(['metadata', 'total_pages', 'Creator', 'Producer', 'CreationDate', 'ModDate', 'Author', 'ContentTypeId', '_dlc_DocIdItemGuid',\
#            'PADocID', 'MediaServiceImageTags', 'Company', 'file_path', 'source', 'Subject', 'Keywords'], axis=1, inplace=True)
 
display(df1.head(2))

,page_content,metadata,len,page_content_data,source,file_path,page,total_pages,Creator,Producer,...,Author,Title,ContentTypeId,_dlc_DocIdItemGuid,PADocID,MediaServiceImageTags,Subject,Keywords,pagestr,page_content_plus
0,Bendigo Payment FacilitiesTerms...,{'source': '/kaggle/input/benpa...,63,Bendigo Payment Facilities\nTer...,bendigo-payment-facilities-term...,/kaggle/input/benpayfactsandcs/...,0,50,Power PDF Create,Power PDF Create,...,Bianca Pisoni,Bendigo-payment-facilities-term...,0x010100E809092CA74C4245B60BDAE...,da16ccf0-5d9b-4890-8741-29ad993...,39175925v1,,personal-payment-facilities-ter...,personal-payment-facilities-ter...,0,Bendigo Payment FacilitiesTerms...
1,ContentsBendigo Payment Facilit...,{'source': '/kaggle/input/benpa...,885,Contents\nBendigo Payment Facil...,bendigo-payment-facilities-term...,/kaggle/input/benpayfactsandcs/...,1,50,Power PDF Create,Power PDF Create,...,Bianca Pisoni,Bendigo-payment-facilities-term...,0x010100E809092CA74C4245B60BDAE...,da16ccf0-5d9b-4890-8741-29ad993...,39175925v1,,personal-payment-facilities-ter...,personal-payment-facilities-ter...,1,ContentsBendigo Payment Facilit...


In [78]:
# --- Embedd and store (once RAG working, replace storage from df to cloudSQL)
pd.set_option('display.max_colwidth', 35)
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# Load the Sentence Transformer model
#model_trans = SentenceTransformer("paraphrase-mpnet-base-v2")
model_trans = SentenceTransformer('sentence-transformers/all-mpnet-base-v2') # supposedly more powerful that above

# Embed the chunked text
embeddings = model_trans.encode(df1['page_content'])

# Convert embedding vectors into one-dimensional arrays
one_dimensional_embeddings = []
for embedding in embeddings:
    one_dimensional_embedding = np.ravel(embedding)
    one_dimensional_embeddings.append(one_dimensional_embedding)

# Combine the original text and embeddings into a DataFrame
data = {
    "original_text": df1['page_content'],
    "embeddings": one_dimensional_embeddings
}

dfe = pd.DataFrame(data)
df3 = pd.concat([df1, dfe], axis=1)
df3.drop(['original_text'], axis=1, inplace=True)
display(df3.head(3))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

,page_content,metadata,len,page_content_data,source,file_path,page,total_pages,Creator,Producer,...,Title,ContentTypeId,_dlc_DocIdItemGuid,PADocID,MediaServiceImageTags,Subject,Keywords,pagestr,page_content_plus,embeddings
0,Bendigo Payment FacilitiesTerms...,{'source': '/kaggle/input/benpa...,63,Bendigo Payment Facilities\nTer...,bendigo-payment-facilities-term...,/kaggle/input/benpayfactsandcs/...,0,50,Power PDF Create,Power PDF Create,...,Bendigo-payment-facilities-term...,0x010100E809092CA74C4245B60BDAE...,da16ccf0-5d9b-4890-8741-29ad993...,39175925v1,,personal-payment-facilities-ter...,personal-payment-facilities-ter...,0,Bendigo Payment FacilitiesTerms...,"[-0.009844421, 0.0152423745, 0...."
1,ContentsBendigo Payment Facilit...,{'source': '/kaggle/input/benpa...,885,Contents\nBendigo Payment Facil...,bendigo-payment-facilities-term...,/kaggle/input/benpayfactsandcs/...,1,50,Power PDF Create,Power PDF Create,...,Bendigo-payment-facilities-term...,0x010100E809092CA74C4245B60BDAE...,da16ccf0-5d9b-4890-8741-29ad993...,39175925v1,,personal-payment-facilities-ter...,personal-payment-facilities-ter...,1,ContentsBendigo Payment Facilit...,"[0.014502276, -0.021184146, 0.0..."
2,Bendigo Payment Facilities Term...,{'source': '/kaggle/input/benpa...,920,Bendigo Payment Facilities Term...,bendigo-payment-facilities-term...,/kaggle/input/benpayfactsandcs/...,2,50,Power PDF Create,Power PDF Create,...,Bendigo-payment-facilities-term...,0x010100E809092CA74C4245B60BDAE...,da16ccf0-5d9b-4890-8741-29ad993...,39175925v1,,personal-payment-facilities-ter...,personal-payment-facilities-ter...,2,Bendigo Payment Facilities Term...,"[-0.0067184865, -0.04306116, 0...."


In [ ]:
# Vertex code. Replaced with next cell to run in Kaggle
# --- Embed question (once all running, convert this and most other code to functions (or a class))
# from sentence_transformers import SentenceTransformer
# import numpy as np
# import pandas as pd

# # Initialize the Sentence Transformer model
# model = SentenceTransformer("paraphrase-mpnet-base-v2")

# question = "can the Easy Money Card be used overseas?"

# # Embed the question
# embeddings = model.encode([question])

# # Flatten the embedding list into a one-dimensional array
# one_dimensional_embeddings = np.ravel(embeddings)

# # Convert array to a single-row dataframe
# df_question = pd.DataFrame({'embeddings': [one_dimensional_embeddings]})

# df_question['question'] = question
# df_question = df_question[['question','embeddings']]
# display(df_question)

# Can skip the next 3 as they are combined in the 4th as a single function

In [9]:
# --- Embed question (once all running, convert this and most other code to functions (or a class))
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

# Initialize the Sentence Transformer model
model_trans = SentenceTransformer("paraphrase-mpnet-base-v2")
question = "can the Easy Money Card be used overseas?"

# Embed the question 
# -- (1st line is what i have been using in Vertex)
#question_embedding = model.encode(question, convert_to_tensor=True)
# -- had to replace with this to resolve error
# Solution was to: Encode the question and move the tensor to the CPU
question_embedding = model_trans.encode(question, convert_to_tensor=True).cpu()

# Calculate the similarities
similarities = df3['embeddings'].apply(lambda x: np.dot(question_embedding, x))

# Identify the 3 most similar rows
most_similar_indices = similarities.nlargest(8).index
most_similar_rows = df3.iloc[most_similar_indices]

# Display the most similar rows
display(most_similar_rows)

AttributeError: 'GemmaForCausalLM' object has no attribute 'encode'

In [8]:
# Vertex code
# cont = most_similar_rows['page_content_data'].tolist()
# context = cont

# # ------------------------

# # prompt source: https://www.kaggle.com/code/gpreda/exploring-eu-ai-act-with-gemma/
# prompt = f"""
# You are an AI Agent specialized to answer to questions about the context provided.
# In order to create the answer, please only use the information from the
# context provided (Context). Do not include other information.
# Answer with simple words.
# If needed, include also explanations.
# Question: {question}
# Context: {context}
# Answer:
# """

# instances = [
#     {
#         "prompt": prompt,
#         "max_tokens": 100, #100
#         "temperature": 0.25, #0.15
#         "top_p": 0.025, # 1.0
#         "top_k": 40, #10
#         #"raw_response": False,
#         "raw_response": True,
#     },
# ]
# response = endpoint_vllm.predict(instances=instances)
# prediction = response.predictions[0]
# print(prediction)

NameError: name 'endpoint_vllm' is not defined

In [14]:
from IPython.display import Markdown, display

cont = most_similar_rows['page_content_data'].tolist()
context = cont

#system =  "You are a skilled software engineer who consistently produces high-quality Python code."
system = f"""You are an AI Agent specialized to answer to questions about the context provided.\
In order to create the answer, please only use the information from the\
context provided (Context). Do not include other information.\
Answer with simple words.\
If needed, include also explanations.\
Context: {context}"""

#user = "Write a Python code to display text in a star pattern."
#user = question
user = "can Foreign Currency Transactions performed?"

prompt = f"System: {system} \n User: {user} \n AI: "
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

#outputs = model.generate(**inputs, max_length=1500, temperature = 0.25, num_return_sequences=1)
outputs = model.generate(**inputs, max_length=1500, temperature = 0.25, top_p = 0.025, top_k = 40, \
                         num_return_sequences=1, do_sample=True)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

Markdown(text.split("AI:")[1])

 

Sure, here is the answer to the user's question:

Foreign Currency Transactions performed using your Debit Card which are performed in currencies other than Australian dollars (AUD) are converted to Australian dollars (AUD) by the relevant Card Scheme using its relevant exchange rates and conversion process. As set out in the Schedule of Fees, Charges and Transaction Account Rebates, an International Fee applies to all Foreign Currency Transactions.

# Below is the function that replaces the above 3

In [80]:
# -- Above 3 cells combined to function (but keeping above 3 if I need to work on the separately)

# Above cell, converted to function
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

def answer_question(user, df3):
    question_embedding = model_trans.encode(user, convert_to_tensor=True).cpu()
    similarities = df3['embeddings'].apply(lambda x: np.dot(question_embedding, x))
    
    #most_similar_indices = similarities.nlargest(8).index
    most_similar_indices = similarities.nlargest(4).index
    
    most_similar_rows = df3.iloc[most_similar_indices]

    cont = most_similar_rows['page_content_data'].tolist()
    context = ' '.join(cont)

#     # -- Inference (Vertex version)
#     system = f"""You are an AI Agent specialized to answer to questions about the context provided.\
#     In order to create the answer, please only use the information from the\
#     context provided (Context). Do not include other information.\
#     Answer with simple words.\
#     If needed, include also explanations.\
#     Context: {context}"""
    
    #     # -- Inference (Kaggle 1)
    system = f"""You are an AI Agent specialized to answer to questions about the context provided.\
    In order to create the answer, please only use the information from the\
    context provided (Context). Do not include other information.\
    Context: {context}"""
    
        # -- Inference (KAggle 2)
#     #system = f"""You are an AI Agent specialized to answer to questions about the context provided.\
#     system = f"""You are an AI Agent specialized in finding the passages of text in {context} that most accurately answer questions you receive.\
#                  Your task is then to answer the question using only passages you located and not any other information."""

    #user = "Write a Python code to display text in a star pattern."
    #user = question
    #user = "can Foreign Currency Transactions performed?"

    prompt = f"System: {system} \n User: {user} \n AI: "

    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

    #outputs = model.generate(**inputs, max_length=1500, temperature = 0.25, num_return_sequences=1)
    #outputs = model.generate(**inputs, max_length=1500, temperature = 0.25, top_p = 0.025, top_k = 40, num_return_sequences=1, do_sample=True)
    #outputs = model.generate(**inputs, max_length=1500, temperature = 0.15, top_p = 1.0, top_k = 10, num_return_sequences=1, do_sample=True)
    
    outputs = model.generate(**inputs, max_new_tokens=1500, temperature = 0.10, top_p = 1.0, top_k = 12, num_return_sequences=1, do_sample=True)

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    #Markdown(text.split("AI:")[1])
    out1 = (text.split("AI:")[1])
    return out1
    return context

#user = "who is liable for unauthorised transactions?"
#user = "can Foreign Currency Transactions be performed?"
#user = "How do you process pay anyone payments?"
user = "What happens if accidently entered the incorrect amount for my pay anyone payments?"
print(answer_question(user, df3))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 

If you accidentally entered the incorrect amount for your Pay Anyone payments, there are two possible scenarios:

**1. Excess Payment:**
If the amount you told us to pay was greater than the amount you needed to pay, you must contact the recipient to obtain a refund of the excess. If we processed the Pay Anyone payment as an Osko Payment or a Fast Payment, we may be able to request that the funds, or just the overpaid amount, be returned on your behalf if you ask us to do so. However, the amount will not be returned to you unless the recipient consents to their financial institution returning the funds to us and we receive the returned funds.

**2. Less Payment:**
If the amount you told us to pay was less than the amount you needed to pay, you can make another Pay Anyone payment for the difference.

Please note that these scenarios do not include BPAY payments. If you have made a mistaken BPAY Payment, please refer to the separate section on BPAY Payments below.


In [81]:
# Extract of code from above to view context
question_embedding = model_trans.encode(user, convert_to_tensor=True).cpu()
similarities = df3['embeddings'].apply(lambda x: np.dot(question_embedding, x))
most_similar_indices = similarities.nlargest(4).index
most_similar_rows = df3.iloc[most_similar_indices]
cont = most_similar_rows['page_content_data'].tolist()
context = ' '.join(cont)
print (context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

to you unless the recipient consents to their financial institution returning the funds to us and we
receive the returned funds; or
• the amount you told us to pay was less than the amount you needed to pay, you can make another
Pay Anyone payment for the difference.
You should notify us immediately if you think that:
• you have made another type of mistake when making a Pay Anyone payment;
• you did not authorise a Pay Anyone payment that has been debited to your Account or if you think
a Pay Anyone payment has not been processed in accordance with your instructions;
• you become aware that a Pay Anyone payment made to a PayID from your Account was directed
to an incorrect recipient; or
• you were fraudulently induced to make a Pay Anyone payment.
Where we consider it appropriate and we are reasonably able to do so, we may request that the
financial institution to whom the funds were transferred returns the funds to us, on your behalf.
However, this is not always possible. 7.6 Errors


# -- Experimenting...

### -- Experimenting with knowledge graphs

In [ ]:

# Error: ValueError: The specified LLM does not support the 'with_structured_output'. Please ensure you are using an LLM that supports this feature

# #!pip install langchain-experimental
# from langchain.llms import LlamaCpp
# #llm = LlamaCpp(model_path="path/to/llama/model")
# llm = model

# from langchain_experimental.graph_transformers import LLMGraphTransformer
# llm_transformer = LLMGraphTransformer(llm=llm)

# from langchain.vectorstores import FAISS
# vector_store = FAISS.from_texts(texts)

# • Extract graph data from text using the LLMGraphTransformer:
# from langchain_core.documents import Document
# doc = Document(text="Some text...")
# graph_data = llm_transformer.transform(doc)

# • Store the extracted graph data in the vector store:
# vector_store.add_graph_data(graph_data)

### -- Experimenting with alternate semantic search

In [84]:
# ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
# ERROR: No matching distribution found for faiss

# !pip install faiss
# import faiss
# # Prepare the FAISS index
# embeddings = np.stack(df3['embeddings'].tolist())
# index = faiss.IndexFlatIP(embeddings.shape[1])
# index.add(embeddings)

# # Query the FAISS index
# question_embedding = model_trans.encode(user, convert_to_tensor=True).cpu().numpy()
# distances, indices = index.search(question_embedding[None, :], 4)

# # Retrieve the most similar rows
# most_similar_rows = df3.iloc[indices[0]]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss
